In [282]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [283]:
train = pd.read_csv('./train.csv')

In [284]:
test = pd.read_csv('./test.csv')

In [285]:
train.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,4,2009,WD,Normal,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,1,2010,WD,Abnorml,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,3,2010,WD,Normal,138500


In [286]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 82 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Frontage       1721 non-null float64
Lot Area           2051 non-null int64
Street             2051 non-null object
Alley              140 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         20

In [287]:
test.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [288]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
Id                 879 non-null int64
PID                879 non-null int64
MS SubClass        879 non-null int64
MS Zoning          879 non-null object
Lot Frontage       719 non-null float64
Lot Area           879 non-null int64
Street             879 non-null object
Alley              58 non-null object
Lot Shape          879 non-null object
Land Contour       879 non-null object
Utilities          879 non-null object
Lot Config         879 non-null object
Land Slope         879 non-null object
Neighborhood       879 non-null object
Condition 1        879 non-null object
Condition 2        879 non-null object
Bldg Type          879 non-null object
House Style        879 non-null object
Overall Qual       879 non-null int64
Overall Cond       879 non-null int64
Year Built         879 non-null int64
Year Remod/Add     879 non-null int64
Roof Style         879 non-null object
Roof M

In [289]:
train['Sale Condition'].value_counts()

Normal     1696
Partial     164
Abnorml     132
Family       29
Alloca       19
AdjLand      11
Name: Sale Condition, dtype: int64

In [290]:
abnormality = {'Abnorml': 1, 'Normal': 0, 'Partial': 0, 'Family': 0, 'Alloca': 0, 'AdjLand': 0}
#abnormal to one, not abnormal (i.e., everything else) goes to not one.
train['TargetSaleCond'] = train['Sale Condition'].map(abnormality)
train['TargetSaleCond'].value_counts()

0    1919
1     132
Name: TargetSaleCond, dtype: int64

In [291]:
train['TargetSaleCond'].value_counts(normalize=True)
#produced baseline value

0    0.935641
1    0.064359
Name: TargetSaleCond, dtype: float64

In [292]:
#comment this all out for full dataset
#randomly pulls about one in ten from the original train that have a normal
#sale condition to build more predictive power into the model instead of baseline
#rand = np.random.randint(0, 11, size=190)
#sample = []
#tens = 0
#for r in rand:
#    sample.append(tens*10 + r)
#    tens += 1
#equal_train = train[train['TargetSaleCond'] == 0].copy()
#equal_train['index'] = equal_train.index
#equal_train.set_index(np.arange(0,len(train[train['TargetSaleCond'] == 0])), inplace = True)
#drops = []
#for n in np.arange(0,len(train[train['TargetSaleCond'] == 0])):
#    if n not in sample:
#        drops.append(n)
#equal_train.drop(drops, inplace=True)
#equal_train.set_index('index', inplace=True)
#print(equal_train.shape)
#train = pd.concat([equal_train, train[train['TargetSaleCond'] == 1]])
#this didn't end up working all that well, but I figured I'd keep the code here

In [293]:
#train['TargetSaleCond'].value_counts(normalize=True)
#checking our increased baseline to try and find a more predictive value
#to double check previous method for increasing the abnormal/normal ratio

In [294]:
#dropping Sale Condition and Sale Price, as we've transformed the first and the second is irrelevant for this half
train.drop('Sale Condition', axis=1, inplace=True)
train.drop('SalePrice', axis=1, inplace=True)

In [295]:
#IDs are superfluous for our analysis
train.drop(['Id', 'PID'], axis = 1, inplace=True)
test.drop(['Id', 'PID'], axis = 1, inplace=True)
print(train.shape)
print(test.shape)

(2051, 79)
(879, 78)


In [296]:
#making a function to dummy variable across both data sets to make sure columns are equivalent
def dummy_var(column):
    if train[column].dtype != str:
        train[column] = train[column].astype(str)
        test[column] = test[column].astype(str)
    #in case categorical data is not a str, as MS SubClass is
    dummy = pd.concat([train[column], test[column]])
    # concats train and test columns
    dummy_df = pd.DataFrame(dummy, columns=[column])
    # keeps column name
    dummy_df = pd.get_dummies(dummy_df)
    #gets dummy variables using pandas
    for dummycol in dummy_df.columns:
        #for some reason, still getting NaN dummy variables despite that being defaulted to false
        if 'nan' not in dummycol:
            train[dummycol] = dummy_df[dummycol].iloc[0:len(train)]
            test[dummycol] = dummy_df[dummycol].iloc[len(train):(len(dummy_df))]
        else:
            dummy_df.drop(dummycol, axis=1, inplace=True)
    train.drop(column, axis=1, inplace=True)
    test.drop(column, axis=1, inplace=True)
    #drops the original variables, as they're no longer needed for analysis
    print("Dummied " + column)
    return dummy_df
    #prints as a double-check, then returns
MSSub_dummy = dummy_var('MS SubClass')
#using MS SubClass as an example, as it's the first entry in the dataset and is categorical

Dummied MS SubClass


In [297]:
print(train.shape)
print(test.shape)
train.head()
#lengths remain the same, dummy variables are added

(2051, 94)
(879, 93)


,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90
0,RL,NaN,13517,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,...,0,0,0,0,1,0,0,0,0,0
1,RL,43.0,11492,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,...,0,0,0,0,1,0,0,0,0,0
2,RL,68.0,7922,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,0
3,RL,73.0,9802,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,1,0,0,0,0,0
4,RL,82.0,14235,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,1,0,0,0,0,0,0


In [298]:
test.head()

,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,...,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90
0,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,0
1,RL,NaN,9662,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,1
2,RL,58.0,17104,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,1,0,0,0,0,0
3,RM,60.0,8520,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,1,0,0,0,0,0,0,0,0,0
4,RL,NaN,9500,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,0,0,0,0


In [299]:
#making sure test data is properly matching
MSSub_dummy.iloc[len(train):(len(train) + 5)]

,MS SubClass_120,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS SubClass_20,MS SubClass_30,MS SubClass_40,MS SubClass_45,MS SubClass_50,MS SubClass_60,MS SubClass_70,MS SubClass_75,MS SubClass_80,MS SubClass_85,MS SubClass_90
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [300]:
#make a list of non-numeric data columns, based off the data dictionary
categories = ['MS Zoning', 'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config',
             'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style',
             'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual', 
             'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1',
             'BsmtFin Type 2', 'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual',
             'Functional', 'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond',
             'Paved Drive', 'Pool QC', 'Fence', 'Misc Feature', 'Sale Type']
#dummy them all out
for c in categories:
    dummy_var(c)
print(train.shape)
print(test.shape)

Dummied MS Zoning
Dummied Street
Dummied Alley
Dummied Lot Shape
Dummied Land Contour
Dummied Utilities
Dummied Lot Config
Dummied Land Slope
Dummied Neighborhood
Dummied Condition 1
Dummied Condition 2
Dummied Bldg Type
Dummied House Style
Dummied Roof Style
Dummied Roof Matl
Dummied Exterior 1st
Dummied Exterior 2nd
Dummied Mas Vnr Type
Dummied Exter Qual
Dummied Exter Cond
Dummied Foundation
Dummied Bsmt Qual
Dummied Bsmt Cond
Dummied Bsmt Exposure
Dummied BsmtFin Type 1
Dummied BsmtFin Type 2
Dummied Heating
Dummied Heating QC
Dummied Central Air
Dummied Electrical
Dummied Kitchen Qual
Dummied Functional
Dummied Fireplace Qu
Dummied Garage Type
Dummied Garage Finish
Dummied Garage Qual
Dummied Garage Cond
Dummied Paved Drive
Dummied Pool QC
Dummied Fence
Dummied Misc Feature
Dummied Sale Type
(2051, 314)
(879, 313)


In [301]:
#checking that all the objects have been turned to numbers
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Columns: 314 entries, Lot Frontage to Sale Type_WD 
dtypes: float64(11), int64(25), uint8(278)
memory usage: 1.1 MB


In [302]:
#checking nulls in remaining data sets
train_has_nulls = {}
test_has_nulls = {}
for c in train.columns:
    if train[c].isnull().sum() > 0:
        train_has_nulls[c] = train[c].isnull().sum()
for c in test.columns:
    if test[c].isnull().sum() > 0:
        test_has_nulls[c] = test[c].isnull().sum()
print(train_has_nulls)
print(test_has_nulls)

{'Lot Frontage': 330, 'Mas Vnr Area': 22, 'BsmtFin SF 1': 1, 'BsmtFin SF 2': 1, 'Bsmt Unf SF': 1, 'Total Bsmt SF': 1, 'Bsmt Full Bath': 2, 'Bsmt Half Bath': 2, 'Garage Yr Blt': 114, 'Garage Cars': 1, 'Garage Area': 1}
{'Lot Frontage': 160, 'Mas Vnr Area': 1, 'Garage Yr Blt': 45}


In [303]:
#checking our low number null values
for c in train[train['BsmtFin SF 1'].isnull()].columns:
    if train[train['BsmtFin SF 1'].isnull()][c].isnull().values[0]:
        print(c)
train[train['BsmtFin SF 1'].isnull()]
#source of most of our basement nulls

BsmtFin SF 1
BsmtFin SF 2
Bsmt Unf SF
Total Bsmt SF
Bsmt Full Bath
Bsmt Half Bath


,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
1327,99.0,5940,4,7,1946,1950,0.0,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0


In [304]:
#made need to comment these in/out dependent when sampling less than full data set
#checking if this particular index has anything in other basement values - does not appear so
for c in train.columns:
    if 'Bsmt' in c:
        print(c + " - " + str(train.loc[1327][c]))

BsmtFin SF 1 - nan
BsmtFin SF 2 - nan
Bsmt Unf SF - nan
Total Bsmt SF - nan
Bsmt Full Bath - nan
Bsmt Half Bath - nan
Bsmt Qual_Ex - 0.0
Bsmt Qual_Fa - 0.0
Bsmt Qual_Gd - 0.0
Bsmt Qual_Po - 0.0
Bsmt Qual_TA - 0.0
Bsmt Cond_Ex - 0.0
Bsmt Cond_Fa - 0.0
Bsmt Cond_Gd - 0.0
Bsmt Cond_Po - 0.0
Bsmt Cond_TA - 0.0
Bsmt Exposure_Av - 0.0
Bsmt Exposure_Gd - 0.0
Bsmt Exposure_Mn - 0.0
Bsmt Exposure_No - 0.0
BsmtFin Type 1_ALQ - 0.0
BsmtFin Type 1_BLQ - 0.0
BsmtFin Type 1_GLQ - 0.0
BsmtFin Type 1_LwQ - 0.0
BsmtFin Type 1_Rec - 0.0
BsmtFin Type 1_Unf - 0.0
BsmtFin Type 2_ALQ - 0.0
BsmtFin Type 2_BLQ - 0.0
BsmtFin Type 2_GLQ - 0.0
BsmtFin Type 2_LwQ - 0.0
BsmtFin Type 2_Rec - 0.0
BsmtFin Type 2_Unf - 0.0


In [305]:
#are zeroes unusual in basement data?
train['BsmtFin SF 1'].describe()
#no

count    2050.000000
mean      442.300488
std       461.204124
min         0.000000
25%         0.000000
50%       368.000000
75%       733.750000
max      5644.000000
Name: BsmtFin SF 1, dtype: float64

In [306]:
#given that, setting all missing basement values to 0
col = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath']
for c in col:
    train.loc[1327, c] = 0
train.loc[1327]

Lot Frontage           99.0
Lot Area             5940.0
Overall Qual            4.0
Overall Cond            7.0
Year Built           1946.0
Year Remod/Add       1950.0
Mas Vnr Area            0.0
BsmtFin SF 1            0.0
BsmtFin SF 2            0.0
Bsmt Unf SF             0.0
Total Bsmt SF           0.0
1st Flr SF            896.0
2nd Flr SF              0.0
Low Qual Fin SF         0.0
Gr Liv Area           896.0
Bsmt Full Bath          0.0
Bsmt Half Bath          0.0
Full Bath               1.0
Half Bath               0.0
Bedroom AbvGr           2.0
Kitchen AbvGr           1.0
TotRms AbvGrd           4.0
Fireplaces              0.0
Garage Yr Blt        1946.0
Garage Cars             1.0
Garage Area           280.0
Wood Deck SF            0.0
Open Porch SF           0.0
Enclosed Porch          0.0
3Ssn Porch              0.0
                      ...  
Garage Cond_Fa          0.0
Garage Cond_Gd          0.0
Garage Cond_Po          0.0
Garage Cond_TA          1.0
Paved Drive_N       

In [307]:
#after checking the data, everything's intended to be ints, so changing
basement = ['BsmtFin SF 1','BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']
for b in basement:
    train[basement] = train[basement].astype(int)
    test[basement] = test[basement].astype(int)

In [308]:
for c in train[(train['Bsmt Full Bath'].isnull())].columns:
    if train[train['Bsmt Full Bath'].isnull()][c].isnull().values[0]:
        print(c)
train[train['Bsmt Full Bath'].isnull()]
#source of our other basement nulls

Bsmt Full Bath
Bsmt Half Bath


,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
616,123.0,47007,5,7,1959,1996,0.0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [309]:
#does this place have a basement? if not, might as well set both to 0
print(train.loc[616]['Total Bsmt SF'])
#sure enough, we'll set both missing values to 0
train.loc[616, 'Bsmt Full Bath'] = 0
train.loc[616, 'Bsmt Half Bath'] = 0
train.loc[616]

0.0


Lot Frontage           123.0
Lot Area             47007.0
Overall Qual             5.0
Overall Cond             7.0
Year Built            1959.0
Year Remod/Add        1996.0
Mas Vnr Area             0.0
BsmtFin SF 1             0.0
BsmtFin SF 2             0.0
Bsmt Unf SF              0.0
Total Bsmt SF            0.0
1st Flr SF            3820.0
2nd Flr SF               0.0
Low Qual Fin SF          0.0
Gr Liv Area           3820.0
Bsmt Full Bath           0.0
Bsmt Half Bath           0.0
Full Bath                3.0
Half Bath                1.0
Bedroom AbvGr            5.0
Kitchen AbvGr            1.0
TotRms AbvGrd           11.0
Fireplaces               2.0
Garage Yr Blt         1959.0
Garage Cars              2.0
Garage Area            624.0
Wood Deck SF             0.0
Open Porch SF          372.0
Enclosed Porch           0.0
3Ssn Porch               0.0
                      ...   
Garage Cond_Fa           0.0
Garage Cond_Gd           0.0
Garage Cond_Po           0.0
Garage Cond_TA

In [310]:
#resetting them all to be ints
train['Bsmt Full Bath'] = train['Bsmt Full Bath'].astype(int)
train['Bsmt Half Bath'] = train['Bsmt Half Bath'].astype(int)
test['Bsmt Full Bath'] = test['Bsmt Full Bath'].astype(int)
test['Bsmt Half Bath'] = test['Bsmt Half Bath'].astype(int)

In [311]:
for c in train[train['Garage Cars'].isnull()].columns:
    if train[train['Garage Cars'].isnull()][c].isnull().values[0]:
        print(c)
train[train['Garage Cars'].isnull()]
#source of our garage nulls

Garage Yr Blt
Garage Cars
Garage Area


,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
1712,50.0,9060,5,6,1923,1999,0.0,548,0,311,...,0,0,0,0,0,0,0,0,0,1


In [312]:
#checking how garage years were inputted without garage data
train['Garage Yr Blt'].describe()

count    1937.000000
mean     1978.707796
std        25.441094
min      1895.000000
25%      1961.000000
50%      1980.000000
75%      2002.000000
max      2207.000000
Name: Garage Yr Blt, dtype: float64

In [313]:
#unusual value there... 2207? Presuming input error, double checking to make sure there aren't any more
train[train['Garage Yr Blt'] > 2010]

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
1699,68.0,8298,8,5,2006,2007,NaN,583,0,963,...,0,0,0,0,0,0,1,0,0,0


In [314]:
#assuming intended value was 2007, changing that value
train.loc[1699, 'Garage Yr Blt'] = 2007

In [315]:
#checking the mins, as 1897 seems early, but the numbers checkout
train[train['Garage Yr Blt'] < 1900]

,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,...,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD
410,66.0,8712,4,7,1896,1950,0.0,0,0,756,...,0,0,0,0,0,0,0,0,0,1
1793,60.0,5280,4,7,1895,1950,0.0,0,0,173,...,0,0,0,0,0,0,0,0,0,1


In [316]:
#checking the other year data for any possible messed up inputs
train['Year Built'].describe()

count    2051.000000
mean     1971.708922
std        30.177889
min      1872.000000
25%      1953.500000
50%      1974.000000
75%      2001.000000
max      2010.000000
Name: Year Built, dtype: float64

In [317]:
train['Year Remod/Add'].describe()

count    2051.000000
mean     1984.190151
std        21.036250
min      1950.000000
25%      1964.500000
50%      1993.000000
75%      2004.000000
max      2010.000000
Name: Year Remod/Add, dtype: float64

In [318]:
#both seem fine - back to fixing our original nulls - checking garage area values before imputation
train['Garage Area'].describe()

count    2050.000000
mean      473.671707
std       215.934561
min         0.000000
25%       319.000000
50%       480.000000
75%       576.000000
max      1418.000000
Name: Garage Area, dtype: float64

In [319]:
#seeing if there's a correlation between lacking a garage area and nulls in the 'Garage Yr Blt'
train[train['Garage Area'] == 0]['Garage Yr Blt']
#sure enough, every single one of our nulls comes from this set

28     NaN
53     NaN
65     NaN
79     NaN
101    NaN
103    NaN
114    NaN
120    NaN
134    NaN
136    NaN
139    NaN
162    NaN
235    NaN
256    NaN
260    NaN
264    NaN
276    NaN
334    NaN
356    NaN
382    NaN
406    NaN
409    NaN
426    NaN
437    NaN
445    NaN
457    NaN
486    NaN
530    NaN
542    NaN
555    NaN
        ..
1451   NaN
1464   NaN
1511   NaN
1519   NaN
1528   NaN
1544   NaN
1545   NaN
1563   NaN
1564   NaN
1584   NaN
1594   NaN
1634   NaN
1705   NaN
1711   NaN
1736   NaN
1754   NaN
1783   NaN
1788   NaN
1799   NaN
1814   NaN
1841   NaN
1893   NaN
1927   NaN
1952   NaN
1982   NaN
1991   NaN
2010   NaN
2027   NaN
2039   NaN
2042   NaN
Name: Garage Yr Blt, Length: 113, dtype: float64

In [320]:
#given that, setting garage cars and garage area on missing values
#we'll decide on what to do with garage yr blt soon
train.loc[1712, 'Garage Cars'] = 0
train.loc[1712, 'Garage Area'] = 0
train.loc[1712]

Lot Frontage           50.0
Lot Area             9060.0
Overall Qual            5.0
Overall Cond            6.0
Year Built           1923.0
Year Remod/Add       1999.0
Mas Vnr Area            0.0
BsmtFin SF 1          548.0
BsmtFin SF 2            0.0
Bsmt Unf SF           311.0
Total Bsmt SF         859.0
1st Flr SF            942.0
2nd Flr SF            886.0
Low Qual Fin SF         0.0
Gr Liv Area          1828.0
Bsmt Full Bath          0.0
Bsmt Half Bath          0.0
Full Bath               2.0
Half Bath               0.0
Bedroom AbvGr           3.0
Kitchen AbvGr           1.0
TotRms AbvGrd           6.0
Fireplaces              0.0
Garage Yr Blt           NaN
Garage Cars             0.0
Garage Area             0.0
Wood Deck SF          174.0
Open Porch SF           0.0
Enclosed Porch        212.0
3Ssn Porch              0.0
                      ...  
Garage Cond_Fa          0.0
Garage Cond_Gd          0.0
Garage Cond_Po          0.0
Garage Cond_TA          0.0
Paved Drive_N       

In [321]:
#changing datatypes again
train['Garage Cars'] = train['Garage Cars'].astype(int)
train['Garage Area'] = train['Garage Area'].astype(int)
test['Garage Cars'] = test['Garage Cars'].astype(int)
test['Garage Area'] = test['Garage Area'].astype(int)

In [322]:
#rerunning to make sure that all minor null values are resolved
train_has_nulls = {}
test_has_nulls = {}
for c in train.columns:
    if train[c].isnull().sum() > 0:
        train_has_nulls[c] = train[c].isnull().sum()
for c in test.columns:
    if test[c].isnull().sum() > 0:
        test_has_nulls[c] = test[c].isnull().sum()
print(train_has_nulls)
print(test_has_nulls)

{'Lot Frontage': 330, 'Mas Vnr Area': 22, 'Garage Yr Blt': 114}
{'Lot Frontage': 160, 'Mas Vnr Area': 1, 'Garage Yr Blt': 45}


In [323]:
#checking how Garage Yr Blt is handled in the test data, before trying to impute anything
print(len(test[test['Garage Area'] == 0]['Garage Yr Blt']))
test[test['Garage Area'] == 0]['Garage Yr Blt']
#there's one missing value here, but that doesn't change the fact that the test data
#has no particular setting for Garage Yr Blt when the garage area is equal to 0.
#Meaning we can either drop the column, or figure out some imputation to carry over both data sets

44


29    NaN
45    NaN
66    NaN
68    NaN
106   NaN
110   NaN
114   NaN
145   NaN
153   NaN
157   NaN
181   NaN
219   NaN
231   NaN
232   NaN
257   NaN
283   NaN
306   NaN
315   NaN
321   NaN
338   NaN
342   NaN
351   NaN
360   NaN
396   NaN
483   NaN
497   NaN
528   NaN
530   NaN
537   NaN
556   NaN
567   NaN
592   NaN
597   NaN
611   NaN
616   NaN
619   NaN
630   NaN
675   NaN
699   NaN
712   NaN
780   NaN
804   NaN
814   NaN
860   NaN
Name: Garage Yr Blt, dtype: float64

In [324]:
#Setting it to 0 would totally throw off the scale, as all the other numbers are years between ~1900 and ~2000
#let's check how many garages were built the same year that the house was in the train and test data
print(len(train[train['Year Built'] == train['Garage Yr Blt']]))
print(len(test[test['Year Built'] == test['Garage Yr Blt']]))

1564
663


In [325]:
#the majority of garages were built at the same time as the house
#so we'll impute the null values with the year the house was built
#then convert to int, to make sure it's proper datatype
for l in range(len(train)):
    if train['Garage Yr Blt'].isnull()[l]:
        train.loc[l, 'Garage Yr Blt'] = train.loc[l, 'Year Built']
#indexes = train[train['Garage Yr Blt'].isnull()].index
#for i in range(len(indexes)):
#    train.loc[indexes[i], 'Garage Yr Blt'] = train.loc[indexes[i], 'Year Built']
#this commented out code was for the sampled w/ equal ratio version
for l in range(len(test)):
    if test['Garage Yr Blt'].isnull()[l]:
        test.loc[l, 'Garage Yr Blt'] = test.loc[l, 'Year Built']
train['Garage Yr Blt'] = train['Garage Yr Blt'].astype(int)
test['Garage Yr Blt'] = test['Garage Yr Blt'].astype(int)

In [326]:
#checking remaining nulls
train_has_nulls = {}
test_has_nulls = {}
for c in train.columns:
    if train[c].isnull().sum() > 0:
        train_has_nulls[c] = train[c].isnull().sum()
for c in test.columns:
    if test[c].isnull().sum() > 0:
        test_has_nulls[c] = test[c].isnull().sum()
print(train_has_nulls)
print(test_has_nulls)

{'Lot Frontage': 330, 'Mas Vnr Area': 22}
{'Lot Frontage': 160, 'Mas Vnr Area': 1}


In [327]:
#according to the data dictionary, lot frontage is based off streets, so let's check those dummy variables
streets = train[train['Lot Frontage'].isnull()].loc[:,['Street_Grvl', 'Street_Pave']].copy()
streets
#no anomalies there - everything has a street, so imputing 0 would be an incorrect value

,Street_Grvl,Street_Pave
0,0,1
7,0,1
8,0,1
23,0,1
27,0,1
31,0,1
42,0,1
50,0,1
59,0,1
62,0,1


In [328]:
#lot frontage seems like it'd be related to lot area, so let's check those values
lots = train[train['Lot Frontage'].isnull() != 1].loc[:,['Lot Area', 'Lot Frontage']].copy()
lots['Area/Frontage'] = (lots['Lot Area'] / lots['Lot Frontage'])
lots['Area/Frontage'].describe()

count    1721.000000
mean      140.739106
std        67.752684
min        26.915033
25%       114.923913
50%       129.415584
75%       150.914634
max      1141.306452
Name: Area/Frontage, dtype: float64

In [329]:
test_lots = test[test['Lot Frontage'].isnull() != 1].loc[:,['Lot Area', 'Lot Frontage']].copy()
test_lots['Area/Frontage'] = (test_lots['Lot Area'] / test_lots['Lot Frontage'])
test_lots['Area/Frontage'].describe()

count     719.000000
mean      140.514242
std        75.147861
min        56.300000
25%       115.000000
50%       129.000000
75%       150.000000
max      1434.966667
Name: Area/Frontage, dtype: float64

In [330]:
#for both the train and test data, Lot Area / Lot Frontage is roughly 140 to 1.
#we could probably get more accurate assessments by splitting based on lot shape 
#and lot config, but the transformations between the train and test data
#would likely be different numbers. To be properly compatible, transformations
#between both data sets for imputation must be the exact same, so we'll simplify
for l in range(len(train)):
    if train['Lot Frontage'].isnull()[l]:
        train.loc[l, 'Lot Frontage'] = round(train.loc[l, 'Lot Area'] / 140)
#indexes = train[train['Lot Frontage'].isnull()].index
#for i in range(len(indexes)):
#    train.loc[indexes[i], 'Lot Frontage'] = round(train.loc[indexes[i], 'Lot Area'] / 140)
for l in range(len(test)):
    if test['Lot Frontage'].isnull()[l]:
        test.loc[l, 'Lot Frontage'] = round(test.loc[l, 'Lot Area'] / 140)
train['Lot Frontage'] = train['Lot Frontage'].astype(int)
test['Lot Frontage'] = test['Lot Frontage'].astype(int)

In [331]:
#should be down to one column
train_has_nulls = {}
test_has_nulls = {}
for c in train.columns:
    if train[c].isnull().sum() > 0:
        train_has_nulls[c] = train[c].isnull().sum()
for c in test.columns:
    if test[c].isnull().sum() > 0:
        test_has_nulls[c] = test[c].isnull().sum()
print(train_has_nulls)
print(test_has_nulls)

{'Mas Vnr Area': 22}
{'Mas Vnr Area': 1}


In [332]:
#Mas Vnr Area will likely be related to Mas Vnr Type, so we check those dummy variables
masonry = []
for c in train.columns:
    if 'Mas Vnr Type' in c:
        masonry.append(c)
train[train['Mas Vnr Area'].isnull()].loc[:, masonry].copy()
#all 0s across the board, despite having a None column here. Hmm...

,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone
22,0,0,0,0,0
41,0,0,0,0,0
86,0,0,0,0,0
212,0,0,0,0,0
276,0,0,0,0,0
338,0,0,0,0,0
431,0,0,0,0,0
451,0,0,0,0,0
591,0,0,0,0,0
844,0,0,0,0,0


In [333]:
#checking test as well
test[test['Mas Vnr Area'].isnull()].loc[:, masonry].copy()

,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone
866,0,0,0,0,0


In [334]:
#same deal. So is Mas Vnr Type_None mean no masonry, or none of the above? let's check
print(train[train['Mas Vnr Type_None'] == 1]['Mas Vnr Area'].describe())
print(test[test['Mas Vnr Type_None'] == 1]['Mas Vnr Area'].describe())
#huh. mostly zeroes, but there's some weird outlier variables

count    1218.000000
mean        0.521346
std        12.849932
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       344.000000
Name: Mas Vnr Area, dtype: float64
count    534.000000
mean       1.117978
std       18.269484
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      312.000000
Name: Mas Vnr Area, dtype: float64


In [335]:
print(train[(train['Mas Vnr Type_None'] == 1) & (train['Mas Vnr Area'] > 0)]['Mas Vnr Area'])
print(test[(test['Mas Vnr Type_None'] == 1) & (test['Mas Vnr Area'] > 0)]['Mas Vnr Area'])
#just a couple odd values. I feel safe filling these nulls with 0. 

765       1.0
810     288.0
1148      1.0
1684      1.0
1832    344.0
Name: Mas Vnr Area, dtype: float64
383    285.0
863    312.0
Name: Mas Vnr Area, dtype: float64


In [336]:
train['Mas Vnr Area'].fillna(0, inplace = True)
test['Mas Vnr Area'].fillna(0, inplace = True)
#should be without nulls anymore!
train_has_nulls = {}
test_has_nulls = {}
for c in train.columns:
    if train[c].isnull().sum() > 0:
        train_has_nulls[c] = train[c].isnull().sum()
for c in test.columns:
    if test[c].isnull().sum() > 0:
        test_has_nulls[c] = test[c].isnull().sum()
print(train_has_nulls)
print(test_has_nulls)

{}
{}


In [337]:
#datatype cleaning
train['Mas Vnr Area'] = train['Mas Vnr Area'].astype(int)
test['Mas Vnr Area'] = test['Mas Vnr Area'].astype(int)
train.drop('Mas Vnr Type_None', axis=1, inplace=True)
test.drop('Mas Vnr Type_None', axis=1, inplace=True)
#should drop the 'None' dummy variable here, as it's basically a null dummy variable, which we don't want
for c in train.columns:
    if ('No' in c) | ('NA' in c):
        print(c)
#checking for any other possible null dummy variables, all other variables have purpose
#no basement exposure is different than no basement, for example

Utilities_NoSeWa
Utilities_NoSewr
Neighborhood_NAmes
Neighborhood_NoRidge
Condition 1_Norm
Condition 2_Norm
Bsmt Exposure_No


In [338]:
#double checking everything
train.info()
test.info()
#different storage types thanks to funky dummy variable shenanigans,
#but both are still ints, and that's what's important

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Columns: 313 entries, Lot Frontage to Sale Type_WD 
dtypes: int64(36), uint8(277)
memory usage: 1.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Columns: 312 entries, Lot Frontage to Sale Type_WD 
dtypes: int64(35), uint8(277)
memory usage: 478.2 KB


In [339]:
#with nulls and variables set, we can finally do regression things
#let's select features and all that fun stuff, then scale everything
X = train.copy().drop('TargetSaleCond', axis=1)
y = train['TargetSaleCond']
X_train, X_hold, y_train, y_hold = train_test_split(X, y, stratify=y, random_state=777)
X_test = test.copy()

ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_hold_scaled = ss.transform(X_hold)
X_test_scaled = ss.transform(X_test)

In [342]:
#introduce some classification options
knn = KNeighborsClassifier()
logreg = LogisticRegression()

In [343]:
#let's make a grid search
params = {'n_neighbors': range(3,30), 'p' : [1, 2], 'weights': ['uniform', 'distance']}
gs = GridSearchCV(knn, params)
gs.fit(X_train_scaled, y_train)
print(gs.best_score_)
gs.best_params_

0.936280884265


{'n_neighbors': 8, 'p': 2, 'weights': 'uniform'}

In [344]:
#not much of an improvement, tbh. Logistic?
logreg.fit(X_train_scaled, y_train)
logreg.score(X_hold, y_hold)
#worse than even baseline. Let's stick with knn for now

0.80116959064327486

In [345]:
predict = pd.read_csv('./test.csv')

In [346]:
y_test = gs.predict(X_test_scaled)
predict['Sale Condition'] = y_test

In [347]:
predict['Sale Condition'].unique()
#if it's just 0, it's basically just a baseline model

array([0])

In [126]:
#let's try something different then. We'll use feature select
#but to do so for classification, we need a logistic regression

In [351]:
#we'll start with 30 (roughly 10% of the variables), maybe iterate to see what gets best?
rfe = RFE(LogisticRegression(), n_features_to_select=30)
rfe.fit(X_train_scaled, y_train)

RFE(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
  n_features_to_select=30, step=1, verbose=0)

In [352]:
logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)

0.9278752436647173

In [353]:
#this is close to baseline, so we'll save it
lr_rfe = logreg.predict(X_test_scaled[:,rfe.get_support(True)])

In [214]:
#Grid Search doesn't work with RFE, so we have to iterate
#this takes a while, so we'll do 5 jumps each time
for n in np.arange(5,76,5):
    rfe = RFE(LogisticRegression(), n_features_to_select=n)
    rfe.fit(X_train_scaled, y_train)
    logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
    print(str(n) + ": " + str(logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)))

5: 0.93567251462
10: 0.925925925926
15: 0.923976608187
20: 0.925925925926
25: 0.927875243665
30: 0.927875243665
35: 0.929824561404
40: 0.927875243665
45: 0.927875243665
50: 0.925925925926
55: 0.925925925926
60: 0.923976608187
65: 0.922027290448
70: 0.92007797271
75: 0.92007797271


In [215]:
#lower seems to perform better, so we'll iterate around 5
for n in np.arange(5,10):
    rfe = RFE(LogisticRegression(), n_features_to_select=n)
    rfe.fit(X_train_scaled, y_train)
    logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
    print(str(n) + ": " + str(logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)))

5: 0.93567251462
6: 0.927875243665
7: 0.929824561404
8: 0.929824561404
9: 0.925925925926


In [217]:
for n in np.arange(1,6):
    rfe = RFE(LogisticRegression(), n_features_to_select=n)
    rfe.fit(X_train_scaled, y_train)
    logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
    print(str(n) + ": " + str(logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)))

1: 0.93567251462
2: 0.93567251462
3: 0.93567251462
4: 0.93567251462
5: 0.93567251462


In [356]:
#5 still seems most optimal, so we'll use that
rfe = RFE(LogisticRegression(), n_features_to_select=5)
rfe.fit(X_train_scaled, y_train)
logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)

0.93567251461988299

In [357]:
predict['Sale Condition'] = logreg.predict(X_test_scaled[:,rfe.get_support(True)])

In [358]:
#is it baseline?
predict['Sale Condition'].unique()
#no? cool! we'll try it

array([0, 1])

In [ ]:
predict[['Id', 'Sale Condition']].to_csv('rfe-logistic-model.csv')

In [ ]:
#further submissions are just continued playing with RFE and features numbers
#The 5 features was very close to baseline, but others were a bit more open
#it was just playing around with them trying to get a model that would be
#better than baseline on Kaggle. Lo and behold, there was one that eventually was.

In [241]:
predict[lr_rfe == 1]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
90,687,535454160,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,8,2009,COD,0
146,702,902109140,90,RM,60.0,7596,Pave,Grvl,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2009,COD,0
194,1154,532377020,20,RL,60.0,6360,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,Shed,650,1,2008,COD,0
219,760,903481130,85,RM,86.0,5160,Pave,Grvl,Reg,Lvl,...,160,0,NaN,NaN,NaN,0,7,2009,COD,0
242,1645,527301280,20,RL,NaN,10960,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2007,COD,0
262,1361,903453050,70,RM,60.0,10266,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,5,2008,COD,0
271,1999,902332030,190,C (all),60.0,7200,Pave,NaN,Reg,Lvl,...,99,0,NaN,NaN,NaN,0,11,2007,WD,0
291,1954,535402180,20,RL,75.0,8700,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,2,2007,COD,0
362,2299,923251160,20,RL,124.0,27697,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,11,2007,COD,0
363,211,904300150,50,RL,72.0,10632,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,COD,0


In [365]:
rfe = RFE(LogisticRegression(), n_features_to_select=35)
rfe.fit(X_train_scaled, y_train)
logreg.fit(X_train_scaled[:,rfe.get_support(True)], y_train)
logreg.score(X_hold_scaled[:,rfe.get_support(True)], y_hold)

0.92982456140350878

In [367]:
predict['Sale Condition'] = logreg.predict(X_test_scaled[:,rfe.get_support(True)])
predict[predict['Sale Condition'] == 1]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
90,687,535454160,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,8,2009,COD,1
219,760,903481130,85,RM,86.0,5160,Pave,Grvl,Reg,Lvl,...,160,0,NaN,NaN,NaN,0,7,2009,COD,1
233,170,902102100,50,RM,60.0,4800,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,COD,1
242,1645,527301280,20,RL,NaN,10960,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2007,COD,1
262,1361,903453050,70,RM,60.0,10266,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,5,2008,COD,1
271,1999,902332030,190,C (all),60.0,7200,Pave,NaN,Reg,Lvl,...,99,0,NaN,NaN,NaN,0,11,2007,WD,1
364,605,534278230,90,RL,85.0,11475,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,7,2009,COD,1
410,1250,535305030,20,RL,60.0,7350,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,6,2008,COD,1
758,1557,911102180,50,C (all),52.0,5150,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,5,2008,WD,1
801,682,535452100,20,RL,70.0,7560,Pave,NaN,Reg,Lvl,...,0,0,NaN,MnPrv,NaN,0,5,2009,COD,1


In [369]:
predict[['Id', 'Sale Condition']].to_csv('rfe-logistic-model-11.csv')
#I eventually ran out of submissions